This Source Code Form is subject to the terms of the Mozilla Public License, v. 2.0.

If a copy of the MPL was not distributed with this file, You can obtain one at https://mozilla.org/MPL/2.0/.

# Debug the API

This notebook provides an entry point for debugging the API without having to worry about manually authenticating.

First we import the required libraries and check the connection works. We also obtain tokens.

In [ ]:
import requests
import json
import sys
import importlib
import os
import gzip
from datetime import datetime
from dateutil import parser


AUDIENCE = os.getenv('AUTH0_AUDIENCE')
BASE_URL = os.getenv('INGESTION_BASE_URL')
AUTH0_DOMAIN = os.getenv('INGESTION_AUTH0_DOMAIN')
AUTH0_CLIENT_ID_UPDATER = os.getenv('INGESTION_AUTH0_CLIENT_ID_UPDATER')
AUTH0_CLIENT_SECRET_UPDATER = os.getenv('INGESTION_AUTH0_CLIENT_SECRET_UPDATER')
AUTH0_CLIENT_ID_ADMIN = os.getenv('INGESTION_AUTH0_CLIENT_ID_ADMIN')
AUTH0_CLIENT_SECRET_ADMIN = os.getenv('INGESTION_AUTH0_CLIENT_SECRET_ADMIN')


def log(response):
    to_print = f"{response.status_code}: " if response.status_code != 200 else ""
    if hasattr(response, "content") and response.content is not None and response.content != b"":
        to_print += json.dumps(json.loads(response.content), indent=4)
    else:
        to_print += "<no content>"
    print(to_print)


response = requests.get(f"{BASE_URL}/heartbeat")

print(f"Welcome to {os.getenv('APP_NAME')}!")
log(response)

response = requests.post(
    url=f"https://{AUTH0_DOMAIN}/oauth/token",
    headers={"Content-Type": "application/json"},
    data=f'{{"client_id":"{AUTH0_CLIENT_ID_ADMIN}","client_secret":"{AUTH0_CLIENT_SECRET_ADMIN}","audience":"{AUDIENCE}","grant_type":"client_credentials"}}',
)
admin_token = json.loads(response.content)["access_token"]
print(response)

response = requests.post(
    url=f"https://{AUTH0_DOMAIN}/oauth/token",
    headers={"Content-Type": "application/json"},
    data=f'{{"client_id":"{AUTH0_CLIENT_ID_UPDATER}","client_secret":"{AUTH0_CLIENT_SECRET_UPDATER}","audience":"{AUDIENCE}","grant_type":"client_credentials"}}',
)
updater_token = json.loads(response.content)["access_token"]
print(response)

## GET

In [ ]:
response = requests.get(
    url=f"{BASE_URL}/categories/residents",
    headers={"Authorization": f"Bearer {admin_token}"},
)
log(response)

## POST

In [ ]:
payload = {
    "category_id": "residents",
    "indicator_id": "residents.residents",
    "srid": 3,
    "trid": 2,
    "start_date": "2020-02",
    "duration": 5,
}
response = requests.post(
    url=f"{BASE_URL}/query",
    headers={
        "Content-Type": "application/json",
        "Content-Encoding": "gzip",
        "Authorization": f"Bearer {admin_token}",
    },
    data=gzip.compress(json.dumps(payload, default=str).encode("utf-8")),
)
log(response)

## PATCH

In [ ]:
payload = {
    "srid": 2,
    "label": "Commune",
    "label_fr": "Commune",
    "index": 2,
    "description": "A commune is a second-level administrative division in Haiti.",
    "description_fr": "La commune est une division administrative de deuxième niveau en Haïti.",
    "boundaries": {},
}
response = requests.patch(
    url=f"{BASE_URL}/spatial_resolutions/2",
    headers={
        "Content-Type": "application/json",
        "Content-Encoding": "gzip",
        "Authorization": f"Bearer {admin_token}",
    },
    data=gzip.compress(json.dumps(payload, default=str).encode("utf-8")),
)
log(response)

## DELETE

In [ ]:
response = requests.delete(
    url=f"{BASE_URL}/categories/residents_indicators_people",
    headers={"Authorization": f"Bearer {admin_token}"},
)
log(response)

That's it!